# ch2. 마케팅 문구 생성 프롬프트 엔지니어링

이번 챕터에서는 마케팅 문구를 생성하기 위한 프롬프트를 작성해보겠습니다.

## chatGPT API 사용 설정

In [3]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [10]:
def print_streaming_response(response):
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="")

In [12]:
messages = [{"role": "user", "content": "안녕하세요"}]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    stream=True
)
print_streaming_response(response)

안녕하세요! 저는 AI 어시스턴트입니다. 도움이 필요하신 것이 있나요?

## 프롬프트 엔지니어링

### AI에게 역할부여

광고 문구를 생성하기 위해서 먼저 system message를 이용해서 AI에게 "전문 카피라이터" 라는 역할을 부여하겠습니다.  
그 다음, 유저로부터 프롬프트만 전달받아서 응답을 리턴하도록 함수화 하겠습니다.

In [14]:
def request_chat_completion(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 전문 카피라이터입니다."},
            {"role": "user", "content": prompt}
        ],
        stream=True
    )
    return response

"카누" 라는 커피 브랜드의 광고 문구를 작성해달라는 간단한 프롬프트를 넣어서 텍스트를 생성해보겠습니다.

In [15]:
prompt = "'카누'라는 커피믹스 브랜드의 SNS 마케팅을 위한 카피를 3개 생성해줘"
response = request_chat_completion(prompt)
print_streaming_response(response)

1. 매일 아침, 당신의 일상에 커피의 활력을 불어넣어 보세요! 카누 커피믹스는 신선한 원두와 풍부한 향기로 당신의 하루를 시작합니다. 커피 한 잔으로 피로를 씻어내고, 새로운 에너지로 가득 찬 하루를 보내보세요! ☕️✨ #카누커피믹스 #매일아침에한잔

2. 비몽사몽한 아침, 카누 커피믹스의 아로마가 당신을 깨워줄 거에요. 커피잔 한 모금, 입 안에서 퍼지는 풍미는 마치 작은 즐거움의 파티예요. 지금 바로 카누와 함께 하루를 환기해 보세요! 💫☕️ #카누커피믹스 #하루의시작은카누와함께

3. 바쁜 일상에 지친 당신을 위해 카누 커피믹스가 여유를 선물합니다. 한 잔의 아로마 커피로 몸과 마음을 달래며, 마치 휴가지에서 즐기는 듯한 로맨틱한 순간을 만나보세요. 카누와 함께하는 한 때, 힐링의 시간을 즐겨보세요. 🌴☕️ #카누와여유로운시간 #힐링의순간 #카누커피믹스

결과를 보면 마케팅 문구를 만들어주긴 하지만 지나치게 길고, 재치가 느껴지지 않습니다.  
프롬프트 엔지니어링을 이용해서 우리가 원하는 대로 텍스트를 생성하도록 제어해보겠습니다.

### 프롬프트 생성

마케팅 문구를 생성하고자 하는 제품명과 제품에 대한 설명을 파라미터로 전달받아서 프롬프트를 생성해보겠습니다.  
이처럼 파라미터를 전달받는 경우, AI에게 먼저 수행해야할 지시를 알려주고 구분자를 사용해서 그 사이에 파라미터를 지정해주는 것이 좋습니다.  
지시 다음 부분에 최대 길이나 어떤 느낌이 나면 좋겠다는 걸 적어주세요.

In [16]:
def generate_prompt(product_name, product_desc):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
---
제품명: {product_name}
제품설명: {product_desc}
---
""".strip()
    return prompt

In [17]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스"
)

In [18]:
print(prompt)

제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
---
제품명: 카누
제품설명: 집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스
---


In [20]:
response = request_chat_completion(prompt)
print_streaming_response(response)

"카페의 향기, 집에서 즐기다! 카누로 진정한 아메리카노를 느껴보세요."

### 생성하는 텍스트 길이, 개수 조정

이번에는 생성하는 텍스트의 길이와 개수를 프롬프트로 조정해보겠습니다.  
generate_prompt 함수에 파라미터를 추가해보겠습니다.  
개수의 경우 문구를 n개 생성해달라고 적어주면 잘 반영됩니다.  
텍스트 길이 조정의 경우엔 몇 자 이내로 적어달라는 것 보다는, 몇 단어 이내로 적어달라는 것이 좋으며, 앞에 "반드시"를 붙여주세요.

In [21]:
def generate_prompt(product_name, product_desc, num, max_length):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 {num}개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 {max_length} 단어 이내로 작성해주세요.
---
제품명: {product_name}
제품설명: {product_desc}
---
""".strip()
    return prompt

In [22]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스",
    num=5,
    max_length=7
)

In [23]:
print(prompt)

제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 5개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 7 단어 이내로 작성해주세요.
---
제품명: 카누
제품설명: 집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스
---


In [25]:
response = request_chat_completion(prompt)
print_streaming_response(response)

1. 집에서 즐기는 푹신한 카페 아메, 카누!
2. 모닝 루틴에 첨단 커피 믹스 카누!
3. 프로 바리스타와 함께하는 맛있는 아메리카노, 카누!
4. 거실에서도 완벽한 카페 맛 카누!
5. 커피 마니아들의 선택, 카누로 맛있는 아메리카노 함께해요!

### 키워드 포함

마케팅 문구에 반드시 포함했으면 하는 키워드가 있다면 프롬프트에 포함할 수 있습니다.  
generate_prompt에 파라미터를 추가하고, 프롬프트를 약간 수정해보겠습니다.

In [26]:
def generate_prompt(product_name, product_desc, num, max_length, keywords):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 {num}개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 {max_length} 단어 이내로 작성해주세요.
키워드가 주어질 경우, 반드시 키워드 중 하나를 포함해야 합니다.
---
제품명: {product_name}
제품설명: {product_desc}
키워드: {keywords}
---
""".strip()
    return prompt

In [27]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스",
    num=5,
    max_length=7,
    keywords=["브라질", "풍미", "향기"]
)

In [28]:
response = request_chat_completion(prompt)
print_streaming_response(response)

1. 집에서 브라질을 닮은 향기와 풍미를 만나보세요! (아메리카노)
2. 이제 집에서도 특별한 커피를 즐길 수 있어요. (카누)
3. 브라질의 맛과 향기가 담긴 집에서의 아메리카노 (카누)
4. 카페에서 느끼던 아메리카노의 향기, 집에서도! (브라질)
5. 카페 도착 전, 제 자리에서 브라질을 만날 수 있어요. (통조림)

## 정리

지금까지 마케팅 문구를 생성하기 위한 프롬프트 엔지니어링을 배워봤습니다.  
어떤가요? 굉장히 쉽지 않나요? 내가 결과로 얻고 싶은 텍스트를 자연어로 묘사하고, 파라미터만 쏙쏙 넣어주면 됩니다.  
이제 streamlit을 이용해서 UI를 개발하고, 여기에 마케팅 문구 생성 기능을 적용하겠습니다.